In [1]:
from gasp.fmshp               import shp_to_df
from gasp.pnd.mng             import split_df_inN, merge_df, split_df
from gasp.pnd.mng.geomattr    import pointxy_to_cols
from gasp.pnd.mng.prj         import project_df
from gasp.pnd.prop.feat       import df_geom_type
from gasp.toshp               import df_to_shp
from gasp.fmapi.mapqst        import matrix_route
from gasp.pnd.netanlst.mapqst import rev_geocode_df

In [2]:
#from gasp.pnd.netanlst.mapqst import matrix_route
import datetime

time_a = datetime.datetime.now().replace(microsecond=0)

ORIGENS  = r'C:\gis\compara_api\ex2\escolas_publicas.shp'
DESTINOS = r'C:\gis\compara_api\ex1\psp.shp'
RESULT   = r'C:\gis\compara_api\matriz_cost_od.shp'
EPSG     = 3763

#matrix_route
#cost_od(ORIGENS, DESTINOS, 3763, 3763, RESULT, mode="driving-car")

#time_b = datetime.datetime.now().replace(microsecond=0)

#print time_b - time_a

In [3]:
origens  = shp_to_df( ORIGENS)
destinos = shp_to_df(DESTINOS)

origens  = project_df(origens, 4326)
destinos = project_df(destinos, 4326)
"""
origens = pointxy_to_cols(
    origens, geomCol="geometry",
    colX="longitude", colY="latitude"
); destinos = pointxy_to_cols(
    destinos, geomCol="geometry",
    colX="longitude", colY="latitude"
)
"""

'\norigens = pointxy_to_cols(\n    origens, geomCol="geometry",\n    colX="longitude", colY="latitude"\n); destinos = pointxy_to_cols(\n    destinos, geomCol="geometry",\n    colX="longitude", colY="latitude"\n)\n'

In [4]:
origensWLoc = rev_geocode_df(origens, 4326)
print origensWLoc

    OBJECTID           COD_SIG IDTIPO  \
0          1  4202701045001001    999   
1          2  4201702030001001    999   
2          3  3503703010001001    999   
3          4  1103701026001001    999   
4          5  3001301052001001    999   
5          6  4001302007001001    999   
6          7  4202501006001001    999   
7          8  3503001002001001    999   
8          9  1003201193001001    999   
9         10  0806702008001001    999   
10        11  3306801011001001    999   
11        12  4302215004001001    999   
12        13  1102206001001001    999   
13        14  0702105019001001    999   
14        15  1804904027001001    999   
15        16  0202802016001001    999   
16        17  0202901048001001    999   
17        18  3201301162001001    999   
18        19  2107902001001001    999   
19        20  5100901026001001    999   
20        21  3307901010001001    999   
21        22  4200801122001001    999   
22        23  3900313010001001    999   
23        24  22

In [5]:
destinosWLoc = rev_geocode_df(destinos, 4326)

In [8]:
# Get route distance for each origin
destLocations = destinosWLoc.locations.tolist()

def get_matrix(row):
    paramLoc = [row.locations] + destLocations
    
    row["results"] = matrix_route(paramLoc, option="oneToMany")
    
    return row

result = origensWLoc.apply(lambda x: get_matrix(x), axis=1)

In [23]:
destinosWLoc["old_fid"] = destinosWLoc.index
result["old_fid"] = result.index

In [25]:
mm = []
for row in range(result.shape[0]):
    for col in range(1 + destinosWLoc.shape[0]):
        if not col:
            ll = [result.iloc[row].old_fid, result.iloc[row].old_fid, result.iloc[row].results["distance"][col]]
        else:
            ll = [result.iloc[row].old_fid, destinosWLoc[col - 1].old_fid, result.iloc[row].results["distance"][col]]
        mm.append(ll)

matrix = pandas.DataFrame(mm, columns=["fid_origin", "fid_destin", "cost"])

KeyError: 0

In [22]:
print result.iloc[0].results["distance"]

[0, 1.439, 1.725, 2.514, 2.78, 4.671, 2.327, 4.938, 3.49, 3.764, 2.922, 2.688, 4.478, 4.364, 4.12, 2.003, 6.003, 6.003, 3.542, 3.4, 5.359, 5.359, 5.331, 6.858, 6.739, 6.805, 2.476, 3.621, 2.966, 2.858, 1.501, 7.435, 7.28, 3.748, 2.664, 4.17, 3.458, 2.739, 2.739, 2.739, 2.867, 1.978, 2.498, 5.086, 3.954, 4.298, 4.42, 2.894, 4.064, 4.76, 3.17, 4.144, 4.491, 1.797, 2.901, 2.948, 4.181, 6.354, 2.862, 1.81, 1.725, 3.917]


In [ ]:
# Split destinos em Dfs com 99 linhas

lst_destinos = split_df(destinos, 99)

In [ ]:
row = origens.iloc[0]

In [ ]:
# Para uma origem

postLocations = [{"latlng" : {
    "lat" : row["latitude"], "lng" : row["longitude"]
}}]
for df in lst_destinos:
    __destinos = df.latlng.tolist()
    
    #d = matrix_route(postLocations + __destinos, option="oneToMany")

In [ ]:
locations = postLocations + __destinos
loc = locations[:2]
print loc

In [ ]:
loc[0]['adminArea1'] = "PT"
loc[0]['adminArea1Type'] = "Country"
loc[1]['adminArea1'] = "PT"
loc[1]['adminArea1Type'] = "Country"
loc[0]['adminArea5'] = "Lisboa"
loc[0]['adminArea5Type'] = "City"
loc[1]['adminArea5'] = "Lisboa"
loc[1]['adminArea5Type'] = "City"
print loc

In [ ]:
d = matrix_route(loc, option="oneToMany")

In [ ]:
for k in d:
    print k
    print d[k]
    print "-----------"